In [1]:
# Import required libraries and dependencies

In [2]:
#import keras as krs
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import tensorflow as tf
import sklearn as sk
import pandas as pd
import numpy as np
from pylab import plt, mpl
plt.style.use('seaborn')
mpl.rcParams['savefig.dpi'] = 300
mpl.rcParams['font.family'] = 'serif'
np.set_printoptions(precision=4, suppress=True)
import os
import json
import requests
from dotenv import load_dotenv
%matplotlib inline
from pathlib import Path

In [209]:
def assess_NA(data):
    """
    Returns a pandas dataframe denoting the total number of NA values and the percentage of NA values in each column.
    The column names are noted on the index.
    
    Parameters
    ----------
    data: dataframe
    """
    # pandas series denoting features and the sum of their null values
    null_sum = data.isnull().sum()# instantiate columns for missing data
    total = null_sum.sort_values(ascending=False)
    percent = ( ((null_sum / len(data.index))*100).round(2) ).sort_values(ascending=False)
    
    # concatenate along the columns to create the complete dataframe
    df_NA = pd.concat([total, percent], axis=1, keys=['Number of NA', 'Percent NA'])
    
    # drop rows that don't have any missing data; omit if you want to keep all rows
    df_NA = df_NA[ (df_NA.T != 0).any() ]
    
    return df_NA

# us_balance_annual

In [210]:
us_balance_annual = pd.read_csv(
    Path('us-balance-annual.csv'),
    parse_dates=True,
    infer_datetime_format=True,
    delimiter=';',

)


us_balance_annual.head()

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),...,Short Term Debt,Total Current Liabilities,Long Term Debt,Total Noncurrent Liabilities,Total Liabilities,Share Capital & Additional Paid-In Capital,Treasury Stock,Retained Earnings,Total Equity,Total Liabilities & Equity
0,A,45846,USD,2008,Q4,2008-10-31,2008-12-19,2009-12-21,363000000.0,371000000.0,...,NaN,1.330000e+09,2.125000e+09,3.118000e+09,4448000000,7.416000e+09,-7.470000e+09,2.791000e+09,2.559000e+09,7007000000
1,A,45846,USD,2009,Q4,2009-10-31,2009-12-21,2010-12-20,346000000.0,346000000.0,...,1.000000e+06,1.123000e+09,2.904000e+09,3.975000e+09,5098000000,7.558000e+09,-7.627000e+09,2.760000e+09,2.514000e+09,7612000000
2,A,45846,USD,2010,Q4,2010-10-31,2010-12-20,2011-12-16,347000000.0,353000000.0,...,1.501000e+09,3.083000e+09,2.190000e+09,3.377000e+09,6460000000,7.910000e+09,-8.038000e+09,3.444000e+09,3.236000e+09,9696000000
3,A,45846,USD,2011,Q4,2011-10-31,2011-12-16,2012-12-20,347000000.0,355000000.0,...,2.530000e+08,1.837000e+09,1.932000e+09,2.904000e+09,4741000000,8.271000e+09,-8.535000e+09,4.456000e+09,4.316000e+09,9057000000
4,A,45846,USD,2012,Q4,2012-10-31,2012-12-20,2013-12-19,348000000.0,353000000.0,...,2.500000e+08,1.893000e+09,2.112000e+09,3.458000e+09,5351000000,8.495000e+09,-8.707000e+09,5.505000e+09,5.185000e+09,10536000000


In [211]:
supa_balance = us_balance_annual.drop(['SimFinId', 'Currency'], axis=1)


In [212]:
balance_data = supa_balance.groupby(['Ticker','Fiscal Year']).mean()
balance_data

Shares (Basic)  Shares (Diluted)  \
Ticker Fiscal Year                                     
A      2008            363000000.0       371000000.0   
       2009            346000000.0       346000000.0   
       2010            347000000.0       353000000.0   
       2011            347000000.0       355000000.0   
       2012            348000000.0       353000000.0   
...                            ...               ...   
ZYXI   2015             31271234.0        31271234.0   
       2016             31271234.0        31271234.0   
       2017             32156000.0        33196000.0   
       2018             32503000.0        34043000.0   
       2019             32439000.0        33963000.0   

                    Cash, Cash Equivalents & Short Term Investments  \
Ticker Fiscal Year                                                    
A      2008                                            1.429000e+09   
       2009                                            2.493000e+09   
       2010                                            2.649000e+09   
       2011                                            3.527000e+09   
       2012                                            2.351000e+09   
...                                                             ...   
ZYXI   2015                                            8.000000e+03   
       2016                                            2.470000e+05   
       2017                                            5.565000e+06   
       2018                                            1.012800e+07   
       2019                                            1.404000e+07   

                    Accounts & Notes Receivable   Inventories  \
Ticker Fiscal Year                                              
A      2008                         770000000.0  6.460000e+08   
       2009                         595000000.0  5.520000e+08   
       2010                         869000000.0  7.160000e+08   
       2011                         860000000.0  8.980000e+08   
       2012                         923000000.0  1.014000e+09   
...                                         ...           ...   
ZYXI   2015                           2426000.0  3.050000e+05   
       2016                           3028000.0  1.070000e+05   
       2017                           2185000.0  4.230000e+05   
       2018                           2791000.0  8.370000e+05   
       2019                           5833000.0  2.378000e+06   

                    Total Current Assets  Property, Plant & Equipment, Net  \
Ticker Fiscal Year                                                           
A      2008                 3.182000e+09                      8.240000e+08   
       2009                 3.961000e+09                      8.450000e+08   
       2010                 6.169000e+09                      9.800000e+08   
       2011                 5.569000e+09                      1.006000e+09   
       2012                 4.629000e+09                      1.164000e+09   
...                                  ...                               ...   
ZYXI   2015                 2.766000e+06                      8.010000e+05   
       2016                 3.422000e+06                      5.800000e+05   
       2017                 8.371000e+06                      1.880000e+05   
       2018                 1.432400e+07                      8.190000e+05   
       2019                 2.256600e+07                      8.580000e+05   

                    Long Term Investments & Receivables  \
Ticker Fiscal Year                                        
A      2008                                 206000000.0   
       2009                                 163000000.0   
       2010                                 142000000.0   
       2011                                 117000000.0   
       2012                                 109000000.0   
...                                                 ...   
ZYXI   2015                       

In [9]:
balance_data_cash = balance_data[["Cash, Cash Equivalents & Short Term Investments", 
                                  "Accounts & Notes Receivable", 
                                  'Treasury Stock', 
                                  'Retained Earnings']]

In [214]:
balance_sheet_cash_gold = balance_data_cash.dropna().astype(int)
balance_sheet_cash_gold

Cash, Cash Equivalents & Short Term Investments  \
Ticker Fiscal Year                                                    
A      2008                                              1429000000   
       2009                                             -2147483648   
       2010                                             -2147483648   
       2011                                             -2147483648   
       2012                                             -2147483648   
...                                                             ...   
ZTS    2019                                              1934000000   
ZYXI   2016                                                  247000   
       2017                                                 5565000   
       2018                                                10128000   
       2019                                                14040000   

                    Accounts & Notes Receivable  Treasury Stock  \
Ticker Fiscal Year                                                
A      2008                           770000000     -2147483648   
       2009                           595000000     -2147483648   
       2010                           869000000     -2147483648   
       2011                           860000000     -2147483648   
       2012                           923000000     -2147483648   
...                                         ...             ...   
ZTS    2019                          1086000000     -2042000000   
ZYXI   2016                             3028000               0   
       2017                             2185000         -243000   
       2018                             2791000        -3675000   
       2019                             5833000        -3846000   

                    Retained Earnings  
Ticker Fiscal Year                     
A      2008               -2147483648  
       2009               -2147483648  
       2010               -2147483648  
       2011               -2147483648  
       2012               -2147483648  
...                               ...  
ZTS    2019               -2147483648  
ZYXI   2016                  -9776000  
       2017                  -2411000  
       2018                   4864000  
       2019                  14356000  

[8612 rows x 4 columns]

In [231]:
balance_sheet_tanglibles = balance_data[["Property, Plant & Equipment, Net",
                           "Retained Earnings", 
                           "Total Equity"]]

balane_sheet_tanglible_gold = balance_sheet_tanglibles.dropna().astype(int)

In [232]:
balane_sheet_tanglible_gold

Property, Plant & Equipment, Net  Retained Earnings  \
Ticker Fiscal Year                                                        
A      2008                                824000000        -2147483648   
       2009                                845000000        -2147483648   
       2010                                980000000        -2147483648   
       2011                               1006000000        -2147483648   
       2012                               1164000000        -2147483648   
...                                              ...                ...   
ZYXI   2015                                   801000           -9845000   
       2016                                   580000           -9776000   
       2017                                   188000           -2411000   
       2018                                   819000            4864000   
       2019                                   858000           14356000   

                    Total Equity  
Ticker Fiscal Year                
A      2008          -2147483648  
       2009          -2147483648  
       2010          -2147483648  
       2011          -2147483648  
       2012          -2147483648  
...                          ...  
ZYXI   2015             -4071000  
       2016             -3802000  
       2017              4902000  
       2018              9291000  
       2019             19653000  

[18190 rows x 3 columns]

In [221]:
balance_sheet_tanglibles = balance_data[
    ['Treasury Stock', 
     'Long Term Investments & Receivables', 
     'Shares (Basic)', 
     'Shares (Diluted)']]

balance_intan_gold = balance_sheet_tanglibles.dropna().astype(int)

In [222]:
balance_intan_gold = balance_sheet_tanglibles.dropna().astype(int)

In [223]:
balance_intan_gold

Treasury Stock  Long Term Investments & Receivables  \
Ticker Fiscal Year                                                        
A      2008            -2147483648                            206000000   
       2009            -2147483648                            163000000   
       2010            -2147483648                            142000000   
       2011            -2147483648                            117000000   
       2012            -2147483648                            109000000   
...                            ...                                  ...   
ZEN    2014                -652000                              9205000   
       2015                -652000                             22336000   
       2016                -652000                             75168000   
ZNGA   2011             -282897000                            110098000   
       2012             -295113000                            367543000   

                    Shares (Basic)  Shares (Diluted)  
Ticker Fiscal Year                                    
A      2008              363000000         371000000  
       2009              346000000         346000000  
       2010              347000000         353000000  
       2011              347000000         355000000  
       2012              348000000         353000000  
...                            ...               ...  
ZEN    2014               53571000          53571000  
       2015               84926000          84926000  
       2016               93161000          93161000  
ZNGA   2011              288599000         288599000  
       2012              741177000         741177000  

[2621 rows x 4 columns]

In [195]:
balance_data_neg = balance_data[
    ["Inventories",
     "Share Capital & Additional Paid-In Capital", 
     "Payables & Accruals", "Total Noncurrent Liabilities", 
     "Long Term Debt", 
     'Total Liabilities & Equity', 
     'Total Current Liabilities']]

balance_sheet_neg_gold = balance_data_neg.dropna().astype(int)

In [225]:
balance_sheet_neg_gold

Inventories  Share Capital & Additional Paid-In Capital  \
Ticker Fiscal Year                                                            
A      2008           646000000                                 -2147483648   
       2009           552000000                                 -2147483648   
       2010           716000000                                 -2147483648   
       2011           898000000                                 -2147483648   
       2012          1014000000                                 -2147483648   
...                         ...                                         ...   
ZYXI   2013             5002000                                     5617000   
       2014             1935000                                     5733000   
       2015              305000                                     5863000   
       2016              107000                                     6063000   
       2018              837000                                     8191000   

                    Payables & Accruals  Total Noncurrent Liabilities  \
Ticker Fiscal Year                                                      
A      2008                   436000000                   -2147483648   
       2009                   307000000                   -2147483648   
       2010                   499000000                   -2147483648   
       2011                   472000000                   -2147483648   
       2012                   461000000                   -2147483648   
...                                 ...                           ...   
ZYXI   2013                     3765000                       2689000   
       2014                     3425000                        324000   
       2015                     3339000                        228000   
       2016                     3976000                        148000   
       2018                     5432000                       2977000   

                    Long Term Debt  Total Liabilities & Equity  \
Ticker Fiscal Year                                               
A      2008             2125000000                 -1582934592   
       2009            -2147483648                  -977934592   
       2010            -2147483648                  1106065408   
       2011             1932000000                   467065408   
       2012             2112000000                  1946065408   
...                            ...                         ...   
ZYXI   2013                 150000                    17221000   
       2014                 311000                     7114000   
       2015                 216000                     3696000   
       2016                 136000                     4091000   
       2018                2967000                    19251000   

                    Total Current Liabilities  
Ticker Fiscal Year                             
A      2008                        1330000000  
       2009                        1123000000  
       2010                       -2147483648  
       2011                        1837000000  
       2012                        1893000000  
...                                       ...  
ZYXI   2013                           9684000  
       2014                           8057000  
       2015                           7539000  
       2016                           7745000  
       2018                           6983000  

[10796 rows x 7 columns]

# us_cashflow_annual_df

In [175]:
us_cashflow_annual_df = pd.read_csv(
    Path('us-cashflow-annual.csv'),
    parse_dates=True,
    infer_datetime_format=True,
    delimiter=';'
)

us_cashflow_annual_df.head()

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),...,Net Cash from Operating Activities,Change in Fixed Assets & Intangibles,Net Change in Long Term Investment,Net Cash from Acquisitions & Divestitures,Net Cash from Investing Activities,Dividends Paid,Cash from (Repayment of) Debt,Cash from (Repurchase of) Equity,Net Cash from Financing Activities,Net Change in Cash
0,A,45846,USD,2008,FY,2008-10-31,2008-12-19,2010-12-20,363000000.0,371000000.0,...,756000000,-140000000.0,150000000.0,-1.720000e+08,-3.850000e+08,NaN,1.600000e+07,-790000000.0,-7.880000e+08,-421000000
1,A,45846,USD,2009,FY,2009-10-31,2009-12-21,2011-12-16,346000000.0,346000000.0,...,408000000,-127000000.0,94000000.0,4.300000e+07,-4.000000e+06,NaN,7.480000e+08,-86000000.0,6.470000e+08,1074000000
2,A,45846,USD,2010,FY,2010-10-31,2010-12-20,2012-12-20,347000000.0,353000000.0,...,718000000,-114000000.0,38000000.0,-1.108000e+09,-1.174000e+09,0.0,7.180000e+08,-112000000.0,6.010000e+08,170000000
3,A,45846,USD,2011,FY,2011-10-31,2011-12-16,2013-12-19,347000000.0,355000000.0,...,1260000000,-170000000.0,16000000.0,-9.700000e+07,1.294000e+09,0.0,-1.500000e+09,-193000000.0,-1.693000e+09,878000000
4,A,45846,USD,2012,FY,2012-10-31,2012-12-20,2014-12-22,348000000.0,353000000.0,...,1228000000,-194000000.0,5000000.0,-2.257000e+09,-2.366000e+09,-104000000.0,1.500000e+08,-72000000.0,-3.500000e+07,-1174000000


In [176]:
dirty_cash_data = us_cashflow_annual_df.drop(['SimFinId', 'Currency'], axis=1)
cash_flow = dirty_cash_data.groupby(['Ticker','Fiscal Year']).mean()
cash_flow

Shares (Basic)  Shares (Diluted)  \
Ticker Fiscal Year                                     
A      2008            363000000.0       371000000.0   
       2009            346000000.0       346000000.0   
       2010            347000000.0       353000000.0   
       2011            347000000.0       355000000.0   
       2012            348000000.0       353000000.0   
...                            ...               ...   
ZYXI   2015             31271234.0        31271234.0   
       2016             31271234.0        31271234.0   
       2017             32156000.0        33196000.0   
       2018             32503000.0        34043000.0   
       2019             32439000.0        33963000.0   

                    Net Income/Starting Line  Depreciation & Amortization  \
Ticker Fiscal Year                                                          
A      2008                     6.930000e+08                  201000000.0   
       2009                    -3.100000e+07                  162000000.0   
       2010                     6.840000e+08                  202000000.0   
       2011                     1.012000e+09                  253000000.0   
       2012                     1.153000e+09                  301000000.0   
...                                      ...                          ...   
ZYXI   2015                    -2.934000e+06                     424000.0   
       2016                     6.900000e+04                     435000.0   
       2017                     7.365000e+06                     286000.0   
       2018                     9.552000e+06                     448000.0   
       2019                     9.492000e+06                     778000.0   

                    Non-Cash Items  Change in Working Capital  \
Ticker Fiscal Year                                              
A      2008             41000000.0               -179000000.0   
       2009            215000000.0                 62000000.0   
       2010           -116000000.0                -52000000.0   
       2011            219000000.0               -224000000.0   
       2012            -50000000.0               -176000000.0   
...                            ...                        ...   
ZYXI   2015              1198000.0                  1653000.0   
       2016               192000.0                  1073000.0   
       2017              1771000.0                 -1162000.0   
       2018                15000.0                  -607000.0   
       2019               152000.0                 -4119000.0   

                    Change in Accounts Receivable  Change in Inventories  \
Ticker Fiscal Year                                                         
A      2008                           -44000000.0            -14000000.0   
       2009                           193000000.0             47000000.0   
       2010                          -166000000.0            -51000000.0   
       2011                            11000000.0           -208000000.0   
       2012                            19000000.0            -52000000.0   
...                                           ...                    ...   
ZYXI   2015                                   NaN                    NaN   
       2016                                   NaN                    NaN   
       2017                                   NaN                    NaN   
       2018                                   NaN                    NaN   
       2019                                   NaN                    NaN   

                    Change in Accounts Payable  Change in Other  \
Ticker Fiscal Year                                                
A      2008                        -21000000.0     -100000000.0   
       2009                         -7000000.0     -171000000.0   
       2010                        113000000.0       52000000.0   
       2011                        -35000000.0        8000000.0   
       2012                        -31000000.0     -

In [177]:
assess_NA(cash_flow)

,Number of NA,Percent NA
Change in Inventories,17876,92.92
Change in Accounts Payable,17818,92.62
Change in Accounts Receivable,17443,90.67
Change in Other,17269,89.77
Net Change in Long Term Investment,11994,62.35
Dividends Paid,9529,49.53
Net Cash from Acquisitions & Divestitures,7787,40.48
Cash from (Repayment of) Debt,2400,12.48
Cash from (Repurchase of) Equity,2281,11.86
Depreciation & Amortization,415,2.16


In [235]:
cash_flow_var = cash_flow[
    ["Change in Working Capital",
     "Change in Accounts Receivable",
     "Change in Inventories", 
     "Change in Accounts Payable"]]

cash_flow_variables_gold = cash_flow_var.dropna().astype(int)

In [236]:
cash_flow_variables_gold

Change in Working Capital  Change in Accounts Receivable  \
Ticker Fiscal Year                                                             
A      2008                        -179000000                      -44000000   
       2009                          62000000                      193000000   
       2010                         -52000000                     -166000000   
       2011                        -224000000                       11000000   
       2012                        -176000000                       19000000   
...                                       ...                            ...   
XYL    2015                          15000000                      -24000000   
       2016                          61000000                       -6000000   
       2017                         141000000                      -79000000   
       2018                        -176000000                     -103000000   
       2019                          59000000                      -23000000   

                    Change in Inventories  Change in Accounts Payable  
Ticker Fiscal Year                                                     
A      2008                     -14000000                   -21000000  
       2009                      47000000                    -7000000  
       2010                     -51000000                   113000000  
       2011                    -208000000                   -35000000  
       2012                     -52000000                   -31000000  
...                                   ...                         ...  
XYL    2015                      23000000                    20000000  
       2016                     -15000000                    61000000  
       2017                      27000000                    50000000  
       2018                     -97000000                    51000000  
       2019                      47000000                    29000000  

[1074 rows x 4 columns]

In [237]:
cash_flow_positive = cash_flow[
    ['Net Cash from Operating Activities',
     'Net Cash from Acquisitions & Divestitures', 
     'Net Cash from Investing Activities', 
     'Cash from (Repayment of) Debt']]

pos_cash_flow_gold = droped_na_cash_flow_pos.dropna().astype(int)

In [238]:
pos_cash_flow_gold

Net Cash from Acquisitions & Divestitures  \
Ticker Fiscal Year                                              
A      2008                                        -172000000   
       2009                                          43000000   
       2010                                       -1108000000   
       2011                                         -97000000   
       2012                                       -2147483648   
...                                                       ...   
ZUMZ   2016                                          -5395000   
ZUO    2017                                         -11420000   
       2018                                           -247000   
       2019                                                 0   
ZYXI   2012                                           -245000   

                    Net Cash from Acquisitions & Divestitures  \
Ticker Fiscal Year                                              
A      2008                                        -172000000   
       2009                                          43000000   
       2010                                       -1108000000   
       2011                                         -97000000   
       2012                                       -2147483648   
...                                                       ...   
ZUMZ   2016                                          -5395000   
ZUO    2017                                         -11420000   
       2018                                           -247000   
       2019                                                 0   
ZYXI   2012                                           -245000   

                    Net Cash from Investing Activities  \
Ticker Fiscal Year                                       
A      2008                                 -385000000   
       2009                                   -4000000   
       2010                                -1174000000   
       2011                                 1294000000   
       2012                                -2147483648   
...                                                ...   
ZUMZ   2016                                  -51515000   
ZUO    2017                                  -16118000   
       2018                                 -121123000   
       2019                                  -29760000   
ZYXI   2012                                   -1584000   

                    Cash from (Repayment of) Debt  
Ticker Fiscal Year                                 
A      2008                              16000000  
       2009                             748000000  
       2010                             718000000  
       2011                           -1500000000  
       2012                             150000000  
...                                           ...  
ZUMZ   2016                                650000  
ZUO    2017                              11607000  
       2018                              -3176000  
       2019                              -2960000  
ZYXI   2012                               2486000  

[10654 rows x 4 columns]

In [241]:
cash_flow_negative = cash_flow[
    ["Dividends Paid", 
     "Cash from (Repurchase of) Equity", 
     "Net Cash from Financing Activities"]]

neg_flow_gold =cash_flow_negative.dropna().astype(int)

In [242]:
neg_flow_gold

Dividends Paid  Cash from (Repurchase of) Equity  \
Ticker Fiscal Year                                                     
A      2010                      0                        -112000000   
       2011                      0                        -193000000   
       2012             -104000000                         -72000000   
       2013             -156000000                        -739000000   
       2014             -176000000                         -12000000   
...                            ...                               ...   
ZTS    2016             -188000000                        -300000000   
       2017             -206000000                        -500000000   
       2018             -243000000                        -698000000   
       2019             -314000000                        -626000000   
ZYXI   2019               -2262000                             50000   

                    Net Cash from Financing Activities  
Ticker Fiscal Year                                      
A      2010                                  601000000  
       2011                                -1693000000  
       2012                                  -35000000  
       2013                                 -554000000  
       2014                                 -117000000  
...                                                ...  
ZTS    2016                                 -903000000  
       2017                                 -251000000  
       2018                                  533000000  
       2019                                 -951000000  
ZYXI   2019                                   -2231000  

[8818 rows x 3 columns]

# income-annual

In [243]:
us_income_annual_df = pd.read_csv(
    Path('us-income-annual.csv'),
    
    parse_dates=True,
    infer_datetime_format=True,
    delimiter=';'
)

us_income_annual_df.head()

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Restated Date,Shares (Basic),Shares (Diluted),...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adj.",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) from Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income (Common)
0,A,45846,USD,2008,FY,2008-10-31,2008-12-19,2010-12-20,363000000.0,371000000.0,...,20000000.0,-10000000.0,815000000,0.0,815000000,-122000000.0,693000000,NaN,693000000,693000000
1,A,45846,USD,2009,FY,2009-10-31,2009-12-21,2011-12-16,346000000.0,346000000.0,...,-40000000.0,-59000000.0,7000000,0.0,7000000,-38000000.0,-31000000,NaN,-31000000,-31000000
2,A,45846,USD,2010,FY,2010-10-31,2010-12-20,2012-12-20,347000000.0,353000000.0,...,-6000000.0,-76000000.0,560000000,132000000.0,692000000,-8000000.0,684000000,NaN,684000000,684000000
3,A,45846,USD,2011,FY,2011-10-31,2011-12-16,2013-12-19,347000000.0,355000000.0,...,-39000000.0,-72000000.0,1032000000,NaN,1032000000,-20000000.0,1012000000,NaN,1012000000,1012000000
4,A,45846,USD,2012,FY,2012-10-31,2012-12-20,2014-12-22,348000000.0,353000000.0,...,-76000000.0,-92000000.0,1043000000,NaN,1043000000,110000000.0,1153000000,NaN,1153000000,1153000000


In [244]:
dirty_income = us_income_annual_df.drop(['SimFinId'], axis=1)


In [245]:
income_annual = dirty_income.groupby(['Ticker','Fiscal Year']).mean()

income_annual

Shares (Basic)  Shares (Diluted)       Revenue  \
Ticker Fiscal Year                                                   
A      2008            363000000.0       371000000.0  5.774000e+09   
       2009            346000000.0       346000000.0  4.481000e+09   
       2010            347000000.0       353000000.0  5.444000e+09   
       2011            347000000.0       355000000.0  6.615000e+09   
       2012            348000000.0       353000000.0  6.858000e+09   
...                            ...               ...           ...   
ZYXI   2015             31271234.0        31271234.0  1.164100e+07   
       2016             31271234.0        31271234.0  1.331300e+07   
       2017             32156000.0        33196000.0  2.343200e+07   
       2018             32503000.0        34043000.0  3.191700e+07   
       2019             32439000.0        33963000.0  4.547200e+07   

                    Cost of Revenue  Gross Profit  Operating Expenses  \
Ticker Fiscal Year                                                      
A      2008           -2.578000e+09  3.196000e+09       -2.401000e+09   
       2009           -2.189000e+09  2.292000e+09       -2.245000e+09   
       2010           -2.514000e+09  2.930000e+09       -2.364000e+09   
       2011           -3.086000e+09  3.529000e+09       -2.458000e+09   
       2012           -3.254000e+09  3.604000e+09       -2.485000e+09   
...                             ...           ...                 ...   
ZYXI   2015           -4.937000e+06  6.704000e+06       -9.185000e+06   
       2016           -3.517000e+06  9.796000e+06       -9.156000e+06   
       2017           -4.819000e+06  1.861300e+07       -9.669000e+06   
       2018           -6.038000e+06  2.587900e+07       -1.550900e+07   
       2019           -8.814000e+06  3.665800e+07       -2.559200e+07   

                    Selling, General & Administrative  Research & Development  \
Ticker Fiscal Year                                                              
A      2008                             -1.697000e+09            -704000000.0   
       2009                             -1.603000e+09            -642000000.0   
       2010                             -1.752000e+09            -612000000.0   
       2011                             -1.809000e+09            -649000000.0   
       2012                             -1.817000e+09            -668000000.0   
...                                               ...                     ...   
ZYXI   2015                             -9.185000e+06                     NaN   
       2016                             -9.156000e+06                     NaN   
       2017                             -9.669000e+06                     NaN   
       2018                             -1.550900e+07                     NaN   
       2019                             -2.559200e+07                     NaN   

                    Depreciation & Amortization  Operating Income (Loss)  \
Ticker Fiscal Year                                                         
A      2008                                 NaN                795000000   
       2009                                 NaN                 47000000   
       2010                                 NaN                566000000   
       2011                                 NaN               1071000000   
       2012                                 NaN               1119000000   
...                                         ...                      ...   
ZYXI   2015                                 NaN                 -2481000   
       2016                                 NaN                   640000   
       2017                                 NaN                  8944000   
       2018                                 NaN                 10370000   
       2019                                 NaN                 11066000   

                    Non-Operating Income (Loss)  Interest Expense, Net  \
Ticker Fiscal Year                        

In [246]:
assess_NA(income_annual)

,Number of NA,Percent NA
Net Extraordinary Gains (Losses),15139,78.62
Research & Development,11650,60.50
Depreciation & Amortization,10699,55.56
Abnormal Gains (Losses),7115,36.95
"Interest Expense, Net",2156,11.20
Cost of Revenue,1838,9.55
"Income Tax (Expense) Benefit, Net",1136,5.90
"Selling, General & Administrative",950,4.93
Revenue,281,1.46
Gross Profit,265,1.38


In [ ]:
income_pos = income_annual[
    ["Gross Profit", 
     "Operating Income (Loss)", 
     "Income (Loss) from Continuing Operations", 
     "Net Income", 
     "Net Income (Common)"]]

income_pos_gold = income_pos.dropna().astype(int)

In [ ]:
income_pos_gold

In [247]:
in_income = income_annual[
    ["Net Extraordinary Gains (Losses)", 
     "Research & Development", 
     "Non-Operating Income (Loss)"]]

in_income_gold = in_income.dropna().astype(int)

In [248]:
in_income_gold

Net Extraordinary Gains (Losses)  Research & Development  \
Ticker Fiscal Year                                                             
A      2013                                509000000              -337000000   
       2014                                317000000              -358000000   
       2015                                -37000000              -330000000   
ABAX   2012                                 -3668000               -13577000   
       2013                                 -2044000               -13647000   
...                                              ...                     ...   
ZGNX   2015                                 67848000               -27860000   
       2016                                 -1021000               -41840000   
       2017                                  -795000               -67449000   
       2018                                  -198000              -100925000   
       2019                                        0              -115639000   

                    Non-Operating Income (Loss)  
Ticker Fiscal Year                               
A      2013                           -93000000  
       2014                          -190000000  
       2015                           -42000000  
ABAX   2012                              253000  
       2013                              994000  
...                                         ...  
ZGNX   2015                            -3030000  
       2016                            -2336000  
       2017                            -1507000  
       2018                            17290000  
       2019                            10173000  

[1336 rows x 3 columns]

In [249]:
income_neg = income_annual[
    ["Cost of Revenue", 
     "Operating Expenses", 
     "Selling, General & Administrative"]]

income_neg_gold = income_neg.dropna().astype(int)

In [250]:
income_neg_gold

Cost of Revenue  Operating Expenses  \
Ticker Fiscal Year                                        
A      2008             -2147483648         -2147483648   
       2009             -2147483648         -2147483648   
       2010             -2147483648         -2147483648   
       2011             -2147483648         -2147483648   
       2012             -2147483648         -2147483648   
...                             ...                 ...   
ZYXI   2015                -4937000            -9185000   
       2016                -3517000            -9156000   
       2017                -4819000            -9669000   
       2018                -6038000           -15509000   
       2019                -8814000           -25592000   

                    Selling, General & Administrative  
Ticker Fiscal Year                                     
A      2008                               -1697000000  
       2009                               -1603000000  
       2010                               -1752000000  
       2011                               -1809000000  
       2012                               -1817000000  
...                                               ...  
ZYXI   2015                                  -9185000  
       2016                                  -9156000  
       2017                                  -9669000  
       2018                                 -15509000  
       2019                                 -25592000  

[16567 rows x 3 columns]

In [251]:
income_annual.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 19255 entries, ('A', 2008) to ('ZYXI', 2019)
Data columns (total 20 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Shares (Basic)                            19052 non-null  float64
 1   Shares (Diluted)                          19052 non-null  float64
 2   Revenue                                   18974 non-null  float64
 3   Cost of Revenue                           17417 non-null  float64
 4   Gross Profit                              18990 non-null  float64
 5   Operating Expenses                        19190 non-null  float64
 6   Selling, General & Administrative         18305 non-null  float64
 7   Research & Development                    7605 non-null   float64
 8   Depreciation & Amortization               8556 non-null   float64
 9   Operating Income (Loss)                   19255 non-null  int64  
 10  Non-Operating I

# shareprices

In [253]:


us_shareprice_df = pd.read_csv(
    Path('us-shareprices-daily.csv'),
    
    parse_dates=True,
    infer_datetime_format=True,
    delimiter=';'
)

us_shareprice_df.head()

,Ticker,SimFinId,Date,Open,Low,High,Close,Adj. Close,Dividend,Volume,Shares Outstanding
0,GOOG,18,2014-03-27,568.00,552.92,568.00,558.46,558.46,NaN,13100,336050831.0
1,GOOG,18,2014-03-28,561.20,558.67,566.43,559.99,559.99,NaN,41100,336050831.0
2,GOOG,18,2014-03-31,566.89,556.93,567.00,556.97,556.97,NaN,10800,674462000.0
3,GOOG,18,2014-04-01,558.71,558.71,568.45,567.16,567.16,NaN,7900,674462000.0
4,GOOG,18,2014-04-02,565.11,562.19,604.83,567.00,567.00,NaN,146700,674462000.0


In [254]:
ticker_data = us_shareprice_df.groupby(['Ticker', 'Date']).mean()
ticker_data

SimFinId   Open    Low   High  Close  Adj. Close  Dividend  \
Ticker Date                                                                     
A      2007-01-03     45846  34.99  34.05  35.48  34.30       22.72       NaN   
       2007-01-04     45846  34.30  33.46  34.60  34.41       22.80       NaN   
       2007-01-05     45846  34.30  34.00  34.40  34.09       22.58       NaN   
       2007-01-08     45846  33.98  33.68  34.08  33.97       22.51       NaN   
       2007-01-09     45846  34.08  33.63  34.32  34.01       22.53       NaN   
...                     ...    ...    ...    ...    ...         ...       ...   
ZYXI   2020-02-11    171401  10.02   9.55  10.02   9.88        9.88       NaN   
       2020-02-12    171401   9.92   9.92  10.50  10.32       10.32       NaN   
       2020-02-13    171401  10.24  10.10  10.36  10.26       10.26       NaN   
       2020-02-14    171401  10.20   9.95  10.33  10.16       10.16       NaN   
       2020-02-18    171401  10.15  10.01  10.34  10.18       10.18       NaN   

                    Volume  Shares Outstanding  
Ticker Date                                     
A      2007-01-03  2574600                 NaN  
       2007-01-04  2073700                 NaN  
       2007-01-05  2676600                 NaN  
       2007-01-08  1557200                 NaN  
       2007-01-09  1386200                 NaN  
...                    ...                 ...  
ZYXI   2020-02-11   196899          32791665.0  
       2020-02-12   261188          32791665.0  
       2020-02-13   120219          32791665.0  
       2020-02-14   199752          32791665.0  
       2020-02-18   290273          32791665.0  

[5982676 rows x 9 columns]

In [255]:
assess_NA(ticker_data)

,Number of NA,Percent NA
Dividend,5937123,99.24
Shares Outstanding,1226448,20.50
Low,8908,0.15
Close,8682,0.15
Open,8669,0.14
High,8477,0.14
Adj. Close,6134,0.10


In [256]:
ticker_data.info


<bound method DataFrame.info of                    SimFinId   Open    Low   High  Close  Adj. Close  Dividend  \
Ticker Date                                                                     
A      2007-01-03     45846  34.99  34.05  35.48  34.30       22.72       NaN   
       2007-01-04     45846  34.30  33.46  34.60  34.41       22.80       NaN   
       2007-01-05     45846  34.30  34.00  34.40  34.09       22.58       NaN   
       2007-01-08     45846  33.98  33.68  34.08  33.97       22.51       NaN   
       2007-01-09     45846  34.08  33.63  34.32  34.01       22.53       NaN   
...                     ...    ...    ...    ...    ...         ...       ...   
ZYXI   2020-02-11    171401  10.02   9.55  10.02   9.88        9.88       NaN   
       2020-02-12    171401   9.92   9.92  10.50  10.32       10.32       NaN   
       2020-02-13    171401  10.24  10.10  10.36  10.26       10.26       NaN   
       2020-02-14    171401  10.20   9.95  10.33  10.16       10.16       NaN

In [257]:
ticktock_ticker = ticker_data[["Open", "Close", "Adj. Close", "Volume", "Shares Outstanding"]]

In [258]:
ticktock_ticker

Open  Close  Adj. Close   Volume  Shares Outstanding
Ticker Date                                                             
A      2007-01-03  34.99  34.30       22.72  2574600                 NaN
       2007-01-04  34.30  34.41       22.80  2073700                 NaN
       2007-01-05  34.30  34.09       22.58  2676600                 NaN
       2007-01-08  33.98  33.97       22.51  1557200                 NaN
       2007-01-09  34.08  34.01       22.53  1386200                 NaN
...                  ...    ...         ...      ...                 ...
ZYXI   2020-02-11  10.02   9.88        9.88   196899          32791665.0
       2020-02-12   9.92  10.32       10.32   261188          32791665.0
       2020-02-13  10.24  10.26       10.26   120219          32791665.0
       2020-02-14  10.20  10.16       10.16   199752          32791665.0
       2020-02-18  10.15  10.18       10.18   290273          32791665.0

[5982676 rows x 5 columns]

In [259]:
good_list = [balance_data_cash_gold,
             income_pos_gold,
             cash_flow_positive,
             balane_sheet_tanglible_gold]
    

In [260]:
good_list

[                    Cash, Cash Equivalents & Short Term Investments  \
 Ticker Fiscal Year                                                    
 A      2008                                              1429000000   
        2009                                             -2147483648   
        2010                                             -2147483648   
        2011                                             -2147483648   
        2012                                             -2147483648   
 ...                                                             ...   
 ZTS    2019                                              1934000000   
 ZYXI   2016                                                  247000   
        2017                                                 5565000   
        2018                                                10128000   
        2019                                                14040000   
 
                     Accounts & Notes Receivable  Treasury Sto

In [261]:
bad_list = [balance_data_neg,
            income_neg,
            cash_flow_negative
           ]

In [262]:
bad_list

[                     Inventories  Share Capital & Additional Paid-In Capital  \
 Ticker Fiscal Year                                                             
 A      2008         6.460000e+08                                7.416000e+09   
        2009         5.520000e+08                                7.558000e+09   
        2010         7.160000e+08                                7.910000e+09   
        2011         8.980000e+08                                8.271000e+09   
        2012         1.014000e+09                                8.495000e+09   
 ...                          ...                                         ...   
 ZYXI   2015         3.050000e+05                                5.863000e+06   
        2016         1.070000e+05                                6.063000e+06   
        2017         4.230000e+05                                7.645000e+06   
        2018         8.370000e+05                                8.191000e+06   
        2019         2.37800

In [263]:
best_list = [balane_sheet_tanglibles, in_income, cash_flow_var
    
]

In [264]:
best_list

[                    Property, Plant & Equipment, Net  Retained Earnings  \
 Ticker Fiscal Year                                                        
 A      2008                             8.240000e+08       2.791000e+09   
        2009                             8.450000e+08       2.760000e+09   
        2010                             9.800000e+08       3.444000e+09   
        2011                             1.006000e+09       4.456000e+09   
        2012                             1.164000e+09       5.505000e+09   
 ...                                              ...                ...   
 ZYXI   2015                             8.010000e+05      -9.845000e+06   
        2016                             5.800000e+05      -9.776000e+06   
        2017                             1.880000e+05      -2.411000e+06   
        2018                             8.190000e+05       4.864000e+06   
        2019                             8.580000e+05       1.435600e+07   
 
          